<a href="https://colab.research.google.com/github/adrielsvr/Sentimen-Analisis-Publik-terhadap-Transportasi-Kereta-di-Jakarta/blob/main/Preprocessing/Preprocessing_Transportasi_Kereta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalasi library yang dibutuhkan
!pip install nltk
!pip install sastrawi
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from google.colab import drive
import requests
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load dataset twitter mengenai KRL, MRT dan LRT
load_krl= pd.read_excel('/content/drive/MyDrive/Tugas Akhir/Dataset Twitter/Dataset/Data_KRL23.xlsx')
load_mrt = pd.read_excel('/content/drive/MyDrive/Tugas Akhir/Dataset Twitter/Dataset/Data_MRT23.xlsx')
load_lrt = pd.read_excel('/content/drive/MyDrive/Tugas Akhir/Dataset Twitter/Dataset/Data_LRT23.xlsx')

data_krl = pd.DataFrame(load_krl['full_text'])
data_mrt = pd.DataFrame(load_mrt['full_text'])
data_lrt = pd.DataFrame(load_lrt['full_text'])

print(data_lrt.head())
print(data_mrt.head())
print(data_krl.head())

                                           full_text
0  LRT Jabodebek siap mengatasi kemacetan. Yuk ce...
1  Dewi Perssik Sebut Nikita Mirzani Jualan Apem ...
2  Viral Rok Model Melorot, Rachel Vennya Tampil ...
3  Kebangun abis mimpi naik LRT jakarta nyampe di...
4  Saat @erickthohir mendampingi presiden Jokowi ...
                                           full_text
0  Ada Perubahan Jadwal Operasional MRT Jakarta S...
1  @UGM_FESS 4. Lebih bagus lagi kalo bisa pelaja...
2  https://t.co/qCPONlISDS Bali needs airport tra...
3  @WatchmenID Terlalu fokus dengan perang, bom d...
4  siapa disini yg cita-cita nya strolling arroun...
                                           full_text
0  @miaetsumi Hai, Kak. Untuk kartu KRL Commuter ...
1  Keujanan di Bogor, menggigil selama di KRL, pa...
2  @tanyarlfes Karena aku bkn orang jakarta, tiap...
3  @adriansyahyasin Justru di hari lebaran itu KR...
4  @JakartaAwayDay Udah gak ada ruang lagi ah di ...


In [4]:
# Mendefenisikan fungsi untuk cleaning & case folding
def clean_text(text):
    # Menghapus mention
    text = re.sub(r"@\w+", "", text)
    # Menghapus karakter khusus
    text = re.sub(r"[^\w\s]", "", text)
    # Menghapus semua karakter non-word
    text = re.sub(r'\W', ' ', text)
    # Menghapus tautan
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    # Menghapus hashtag
    text = re.sub(r"#\w+", "", text)
    # Menghapus emoji
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                               u"\U0001F300-\U0001F5FF"  # Simbol & Tanda
                               u"\U0001F680-\U0001F6FF"  # Transportasi & Simbol Transportasi
                               u"\U0001F1E0-\U0001F1FF"  # Bendera
                               u"\U00002500-\U00002BEF"  # Simbol & Karakter Aksara CJK
                               u"\U00002702-\U000027B0"  # Simbol, Tanda, & Karakter Aksara Dingbat
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    # Menghapus whitespace tambahan
    text = re.sub(r"\s+", " ", text)
    # Menghapus angka
    text = re.sub(r"\d+", "", text)
    # Menormalisasi huruf kecil
    text = text.lower()

    return text

# Mengimplementasikan fungsi clean_text ke dataset
data_krl['clean_text'] = data_krl['full_text'].apply(clean_text)
data_mrt['clean_text'] = data_mrt['full_text'].apply(clean_text)
data_lrt['clean_text'] = data_lrt['full_text'].apply(clean_text)

# Menampilkan hasil dataset clean_text
print(f"DATA KRL \n {data_krl.head()}\n")
print(f"DATA MRT \n {data_mrt.head()}\n")
print(f"DATA LRT \n {data_lrt.head()}")

DATA KRL 
                                            full_text  \
0  @miaetsumi Hai, Kak. Untuk kartu KRL Commuter ...   
1  Keujanan di Bogor, menggigil selama di KRL, pa...   
2  @tanyarlfes Karena aku bkn orang jakarta, tiap...   
3  @adriansyahyasin Justru di hari lebaran itu KR...   
4  @JakartaAwayDay Udah gak ada ruang lagi ah di ...   

                                          clean_text  
0   hai kak untuk kartu krl commuter line dapat d...  
1  keujanan di bogor menggigil selama di krl pas ...  
2   karena aku bkn orang jakarta tiap kesana seru...  
3   justru di hari lebaran itu krl sepi karena ja...  
4   udah gak ada ruang lagi ah di jakarta mah rib...  

DATA MRT 
                                            full_text  \
0  Ada Perubahan Jadwal Operasional MRT Jakarta S...   
1  @UGM_FESS 4. Lebih bagus lagi kalo bisa pelaja...   
2  https://t.co/qCPONlISDS Bali needs airport tra...   
3  @WatchmenID Terlalu fokus dengan perang, bom d...   
4  siapa disini yg cita-cita n

In [28]:
# Melakukan tokenisasi
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [29]:
# Mendefenisikan fungsi tokenisasi
def word_tokenize(text):
  text = nltk.word_tokenize(text)
  return text

In [30]:
# Mengimplementasikan fungsi tokenisasi ke dataset
data_krl['tokenized_text'] = data_krl['clean_text'].apply(word_tokenize)
data_mrt['tokenized_text'] = data_mrt['clean_text'].apply(word_tokenize)
data_lrt['tokenized_text'] = data_lrt['clean_text'].apply(word_tokenize)

# Menampilkan hasil dataset tokenized_text
print(f"DATA KRL \n {data_krl[['clean_text', 'tokenized_text']].head()}\n")
print(f"DATA MRT \n {data_mrt[['clean_text', 'tokenized_text']].head()}\n")
print(f"DATA LRT \n {data_lrt[['clean_text', 'tokenized_text']].head()}")

DATA KRL 
                                           clean_text  \
0   hai kak untuk kartu krl commuter line dapat d...   
1  keujanan di bogor menggigil selama di krl pas ...   
2   karena aku bkn orang jakarta tiap kesana seru...   
3   justru di hari lebaran itu krl sepi karena ja...   
4   udah gak ada ruang lagi ah di jakarta mah rib...   

                                      tokenized_text  
0  [hai, kak, untuk, kartu, krl, commuter, line, ...  
1  [keujanan, di, bogor, menggigil, selama, di, k...  
2  [karena, aku, bkn, orang, jakarta, tiap, kesan...  
3  [justru, di, hari, lebaran, itu, krl, sepi, ka...  
4  [udah, gak, ada, ruang, lagi, ah, di, jakarta,...  

DATA MRT 
                                           clean_text  \
0  ada perubahan jadwal operasional mrt jakarta s...   
1    lebih bagus lagi kalo bisa pelajari halte yg...   
2                          bali needs airport train    
3   terlalu fokus dengan perang bom dan aksi laga...   
4  siapa disini yg citacita ny

In [31]:
url_slang = 'https://raw.githubusercontent.com/adeariniputri/text-preprocesing/master/slang.csv'
slang = pd.read_csv(url_slang)
kata_benar = {
    "keujanan": "kehujanan", "kesana": "ke sana", "anget": "hangat", "bkn": "bukan", "gitu": "begitu", "gaada": "tidak ada",
    "bareng": "bersama", "dipotongpotong": "potong", "kayak": "seperti", "moga": "semoga", "tiap": "setiap", "pureblood": "darah murni",
    "rmh": "rumah", "gaakan": "tidak akan", "jalanyangjauhjanganlupapulang": "jalan yang jauh jangan lupa pulang",
    "sunscreen": "tabir surya", "perhatiin": "perhati", "saranin": "saran", "datengin": "datang", "nyari": "cari", "cobain": "coba",
    "temanteman": "teman", "nyampe": "sampai", "balik": "kembali", "edition": "edisi", "dislike": "tidak suka", "overrated": "tinggi",
    "still": "masih", "visit": "kunjung", "gin": "gini", "gamau": "tidak mau", "desekan": "desak", "cm": "cuma", "ojol": "ojek online",
    "stressnya": "stres", "info": "informasi", "rb": "ribu", "rela": "rela", "beritaterkini": "berita terkini", "mutualan": "mutual",
    "bukabukaan": "buka", "nyampe": "sampai", "ngandelin": "andal", "mesti": "harus", "deket": "dekat", "nyobain": "coba", "slain": "selain",
    "made": "buatan", "railway": "rel kereta", "knapa": "kenapa", "covering": "liput", "expanse": "luas", "endeavors": "upaya",
    "foster": "mendorong", "traffic": "lalu lintas", "pollution": "polusi", "transforming": "ubah", "system": "sistem", "tackle": "mengatasi",
    "formally": "resmi", "introduced": "kenal", "greater": "lebih besar", "light": "ringan", "rail": "rel", "aimed": "tuju", "countering": "lawan",
    "congestion": "macet", "environmental": "lingkungan", "ngelihat": "lihat", "rangkainnya": "rangkai", "berpindahpindah": "pindah",
    "tarifintegrasi": "tarif integrasi", "gede": "besar", "kenceng": "kencang", "pekerjamigranterlindungi": "pekerja migran terlindungi",
    "aniesberjuang": "Anies berjuang", "pakeajadulu": "pakai saja dulu", "cs": "customer service"
}

slang_dict = dict(zip(slang['slang'], slang['formal']))
slang_dict.update(kata_benar)

def normalize_text(tokens):
    normalized_tokens = [slang_dict.get(word, word) for word in tokens]
    return normalized_tokens

data_krl['normalisasi'] = data_krl['tokenized_text'].apply(lambda x: normalize_text(x))
data_mrt['normalisasi'] = data_mrt['tokenized_text'].apply(lambda x: normalize_text(x))
data_lrt['normalisasi'] = data_lrt['tokenized_text'].apply(lambda x: normalize_text(x))

print(f"DATA KRL \n {data_krl.head()}\n")
print(f"DATA MRT \n {data_mrt.head()}\n")
print(f"DATA LRT \n {data_lrt.head()}")

DATA KRL 
                                            full_text  \
0  @miaetsumi Hai, Kak. Untuk kartu KRL Commuter ...   
1  Keujanan di Bogor, menggigil selama di KRL, pa...   
2  @tanyarlfes Karena aku bkn orang jakarta, tiap...   
3  @adriansyahyasin Justru di hari lebaran itu KR...   
4  @JakartaAwayDay Udah gak ada ruang lagi ah di ...   

                                          clean_text  \
0   hai kak untuk kartu krl commuter line dapat d...   
1  keujanan di bogor menggigil selama di krl pas ...   
2   karena aku bkn orang jakarta tiap kesana seru...   
3   justru di hari lebaran itu krl sepi karena ja...   
4   udah gak ada ruang lagi ah di jakarta mah rib...   

                                      tokenized_text  \
0  [hai, kak, untuk, kartu, krl, commuter, line, ...   
1  [keujanan, di, bogor, menggigil, selama, di, k...   
2  [karena, aku, bkn, orang, jakarta, tiap, kesan...   
3  [justru, di, hari, lebaran, itu, krl, sepi, ka...   
4  [udah, gak, ada, ruang, lagi, ah

In [32]:
# Melakukan penghapusan stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.get_stop_words()

# Mendefenisikan fungsi stopwords removal
def stopwords_removal(tokens):
  clean_tokens = []
  for token in tokens:
    if token not in stopword:
      clean_tokens.append(token)
  return clean_tokens

# Mengimplementasikan fungsi stopwords_removal ke dataset
data_krl['stopwords_removal'] = data_krl['normalisasi'].apply(stopwords_removal)
data_mrt['stopwords_removal'] = data_mrt['normalisasi'].apply(stopwords_removal)
data_lrt['stopwords_removal'] = data_lrt['normalisasi'].apply(stopwords_removal)

# Menampilkan hasil dataset stopwords_removal
print(f"DATA KRL \n {data_krl[['normalisasi', 'stopwords_removal']].head()}\n")
print(f"DATA MRT \n {data_mrt[['normalisasi', 'stopwords_removal']].head()}\n")
print(f"DATA LRT \n {data_lrt[['normalisasi', 'stopwords_removal']].head()}")

DATA KRL 
                                          normalisasi  \
0  [hai, kakak, untuk, kartu, krl, commuter, line...   
1  [kehujanan, di, bogor, menggigil, selama, di, ...   
2  [karena, aku, bukan, orang, jakarta, setiap, k...   
3  [justru, di, hari, lebaran, itu, krl, sepi, ka...   
4  [sudah, tidak, ada, ruang, lagi, ah, di, jakar...   

                                   stopwords_removal  
0  [hai, kakak, kartu, krl, commuter, line, digun...  
1  [kehujanan, bogor, menggigil, selama, krl, pas...  
2  [aku, bukan, orang, jakarta, ke sana, seru, ka...  
3  [justru, hari, lebaran, krl, sepi, jakarta, le...  
4  [ruang, ah, jakarta, mah, ribut, tawuran, metr...  

DATA MRT 
                                          normalisasi  \
0  [ada, perubahan, jadwal, operasional, mrt, jak...   
1  [lebih, bagus, lagi, kalau, bisa, pelajari, ha...   
2                      [bali, needs, airport, train]   
3  [terlalu, fokus, dengan, perang, bom, dan, aks...   
4  [siapa, disini, yang, citac

In [33]:
data_krl[['normalisasi', 'stopwords_removal']].head()

,normalisasi,stopwords_removal
0,"[hai, kakak, untuk, kartu, krl, commuter, line...","[hai, kakak, kartu, krl, commuter, line, digun..."
1,"[kehujanan, di, bogor, menggigil, selama, di, ...","[kehujanan, bogor, menggigil, selama, krl, pas..."
2,"[karena, aku, bukan, orang, jakarta, setiap, k...","[aku, bukan, orang, jakarta, ke sana, seru, ka..."
3,"[justru, di, hari, lebaran, itu, krl, sepi, ka...","[justru, hari, lebaran, krl, sepi, jakarta, le..."
4,"[sudah, tidak, ada, ruang, lagi, ah, di, jakar...","[ruang, ah, jakarta, mah, ribut, tawuran, metr..."


In [ ]:
data_krl.to_csv('data_krl.csv', index=False)
data_mrt.to_csv('data_mrt.csv', index=False)
data_lrt.to_csv('data_lrt.csv', index=False)

In [ ]:
# Melakukan stem teks
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Mendefenisikan fungsi stemmer
def stemming_text(tokens):
  hasil_stemming = []
  for token in tokens :
    stemming = stemmer.stem(token)
    hasil_stemming.append(stemming)
  stemm = []
  stemm = " ".join(hasil_stemming)
  return stemm

In [ ]:
# Mengimplementasikan fungsi ke dataset KRL dan menyimpan ke direktori gdrive
tqdm.pandas(desc="Progress Stemming Data KRL")
data_krl['stemming'] = data_krl['stopwords_removal'].progress_apply(stemming_text)
data_krl.to_csv('/content/drive/MyDrive/Tugas Akhir/Dataset Twitter/Dataset/Data_KRL_stemming.csv', index=False)

In [ ]:
# Mengimplementasikan fungsi ke dataset KRL dan menyimpan ke direktori gdrive
tqdm.pandas(desc="Progress Stemming Data MRT")
data_mrt['stemming'] = data_mrt['stopwords_removal'].progress_apply(stemming_text)
data_mrt.to_csv('/content/drive/MyDrive/Tugas Akhir/Dataset Twitter/Dataset/Data_MRT_stemming.csv', index=False)

Progress Stemming Data MRT: 100%|██████████| 9552/9552 [26:05<00:00,  6.10it/s]


In [ ]:
# Mengimplementasikan fungsi ke dataset KRL dan menyimpan ke direktori gdrive
tqdm.pandas(desc="Progress Stemming Data LRT")
data_lrt['stemming'] = data_lrt['stopwords_removal'].progress_apply(stemming_text)
data_lrt.to_csv('/content/drive/MyDrive/Tugas Akhir/Dataset Twitter/Dataset/Data_LRT_stemming.csv', index=False)

Progress Stemming Data LRT: 100%|██████████| 2308/2308 [05:39<00:00,  6.79it/s]
